In [ ]:
# Region Of Interest
# 얼굴과 눈 인식하기

In [2]:
from jetbot import Camera
from jetbot import bgr8_to_jpeg

In [3]:
camera = Camera.instance(width=720, height=720) # 영상을 받아들일 카메라 인스턴스 생성

In [ ]:
# 카메라 영상의 프레임 이미지를 얻은 다음, 그 이미지 속에서 검출할 객체 찾기

In [22]:
# 카메라 방향 제어를 위한 Pan/Tilt Servo 인스턴스 생성
from servoserial import ServoSerial
import time

pantilt = ServoSerial() 

# 초기 카메라 방향을 좌우, 상하 중간을 바라보도록 설정
pantilt.Servo_serial_double_control(1, 2200, 2, 2700)
time.sleep(0.5) # 코드 합칠 떄를 대비 동작 시간 대기

serial Open!
245
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\x98\x00\n\x02\n\x8c\x00\n\xf5'


In [23]:
# 검출 영역 좌표와 크기

global face_x, face_y, face_w, face_h

face_x = face_y = face_w = face_h = 0

In [4]:
#카메라 프리뷰 위젯 만들기

import traitlets # 위젯의 속성등을 다루는 라이브러리
import ipywidgets.widgets as widgets # 텍스트 상자, 버튼, 슬라이더, 체크 박스, 드롭다운 메뉴 등 시각화 다루기
from IPython.display import display

face_image = widgets.Image(format='jpeg', width=300, height=300)
display(face_image)

Image(value=b'', format='jpeg', height='300', width='300')

In [10]:
!sudo apt install curl

[sudo] password for jetbot: 


In [ ]:
# curl 설치 후 xml 다운
# -L : 리디렉션 응답 시 처리
# -O : 저장할 파일명으로 리소스 파일명 그대로 사용

In [12]:
!curl -L -O https://github.com/opencv/opencv/raw/master/data/haarcascades/haarcascade_frontalface_default.xml

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  908k  100  908k    0     0   974k      0 --:--:-- --:--:-- --:--:--  974k


In [25]:
!curl -L -O https://github.com/opencv/opencv/raw/master/data/haarcascades/haarcascade_eye.xml

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  333k  100  333k    0     0   484k      0 --:--:-- --:--:-- --:--:-- 9089k


In [14]:
!curl -L -O https://github.com/opencv/opencv/raw/master/data/haarcascades/haarcascade_upperbody.xml

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  767k  100  767k    0     0  1011k      0 --:--:-- --:--:-- --:--:-- 1011k


In [6]:
# HAAR Cascade Classifier 학습 데이터 로딩하기

import cv2

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml') # 정면 얼굴 검출용
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml') # 눈 검출용

In [7]:
while 1:
    frame = camera.value
    frame = cv2.resize(frame, (300, 300)) # 이미지를 학습 데이터와 동일한 크기로 변환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # 흑백 이미지로 변환하여 검출에 사용
    
    faces = face_cascade.detectMultiScale(gray) # 이미지 내에서 신체 검출
    
    if len(faces) > 0: #검출된 내용이 있으면
        (face_x, face_y, face_w, face_h) = faces[0]
        
        # 검출 영역 사각 박스로 표시
        cv2.rectangle(frame, (face_x, face_y), (face_x + face_h, face_y + face_w), (0, 255, 0), 4)
        
        # 눈 검출은 얼굴이 검출된 영역 내부에서만 진행하기 위해 ROI 생성
        roi_gray = gray[face_y : face_y + face_h, face_x : face_x + face_w]
        roi_color = frame[face_y : face_y + face_h, face_x : face_x + face_w]
        
        # 얼굴 안에서 눈을 검출
        eyes = eye_cascade.detectMultiScale(roi_gray)
        
        # 눈이 검출되었다면 눈 위치에 대한 좌표 정보 리턴
        for (ex, ey, ew, eh) in eyes:
            
            # 원본 이미지에 눈의 위치를 표시 (roi_color에 표시하면 원본 이미지에도 표시됨)
            cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (255, 0, 0), 2)
        
    #결과 이미지를 프리뷰 위젯에 표시
    face_image.value = bgr8_to_jpeg(frame)

KeyboardInterrupt: 

In [8]:
camera.stop()